# Neural Machine Translation using Attention Model in Keras

In [126]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

from pickle import load
from numpy.random import rand
from numpy.random import shuffle

from tensorflow.python.keras.preprocessing.text import Tokenizer

from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.models import Model
from keras.layers import LSTM, Input
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint


from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

import random
from tqdm import tqdm
#from babel.dates import format_date
#from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

In [127]:
# load a clean dataset
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

In [128]:
dataset = load_clean_sentences('english-french-both.pkl')
train = load_clean_sentences('english-french-train.pkl')
test = load_clean_sentences('english-french-test.pkl')

In [129]:
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [130]:
# max sentence length
def max_length(lines):
    return max(len(line.split()) for line in lines)

In [131]:
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
fra_tokenizer = create_tokenizer(dataset[:, 1])
fra_vocab_size = len(fra_tokenizer.word_index) + 1
fra_length = max_length(dataset[:, 1])
print('french Vocabulary Size: %d' % fra_vocab_size)
print('french Max Length: %d' % (fra_length))

English Vocabulary Size: 2125
English Max Length: 5
french Vocabulary Size: 4397
french Max Length: 10


In [132]:
eng_tokenizer.index_to_word = dict(zip(eng_tokenizer.word_index.values(), eng_tokenizer.word_index.keys()))
fra_tokenizer.index_to_word = dict(zip(fra_tokenizer.word_index.values(), fra_tokenizer.word_index.keys()))

In [133]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

In [134]:
# one hot encode target sequence
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [135]:
# one hot encode target sequence
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [136]:
# prepare training data
trainX = encode_sequences(fra_tokenizer, fra_length, train[:, 1])
trainX = encode_output(trainX, fra_vocab_size)
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_output(trainY, eng_vocab_size)

In [137]:
print(trainX[9])
print(train[0])

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
['i will return' 'je reviendrai']


In [138]:
print(trainX.shape)
print(trainY.shape)
fra_vocab_size

(9000, 10, 4397)
(9000, 5, 2125)


4397

In [139]:
# prepare validation data
testX = encode_sequences(fra_tokenizer, fra_length, test[:, 1])
testX = encode_output(testX, fra_vocab_size)
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_output(testY, eng_vocab_size)

In [140]:
print(testX.shape)
print(testY.shape)

(1000, 10, 4397)
(1000, 5, 2125)


In [141]:
Tx= fra_length
Ty = eng_length

In [142]:
def softmax(x, axis=1):
    
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [143]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [144]:
def one_step_attention(a, s_prev):
    
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas, a])
    
    return context

In [145]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(eng_vocab_size, activation=softmax)

In [146]:
def model(Tx, Ty, n_a, n_s, fra_vocab_size, eng_vocab_size):

    X = Input(shape=(Tx, fra_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []

    # pre-attention Bi-LSTM
    a = Bidirectional(LSTM(n_a, return_sequences = True))(X)
    
    for t in range(Ty):
    
        context = one_step_attention(a, s)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        out = output_layer(s)
        outputs.append(out)
    
    model = Model(inputs=[X, s0, c0], outputs=outputs)
    
    return model

In [147]:
model = model(Tx, Ty, n_a, n_s, fra_vocab_size, eng_vocab_size)

In [148]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [149]:
m = trainX.shape[0]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(trainY.swapaxes(0,1))
testY = list(testY.swapaxes(0, 1))

In [26]:
# fit model
filename = 'model_.h5'
checkpoint = ModelCheckpoint(filename, monitor='loss', verbose=1, save_best_only=True, mode='min')
model.fit([trainX, s0, c0], outputs, epochs=30, batch_size=100,callbacks=[checkpoint], verbose=2)

Epoch 1/30
 - 48s - loss: 18.8911 - dense_3_loss: 0.3993

Epoch 00001: loss improved from inf to 18.89114, saving model to model_.h5


C:\Python3.5\lib\site-packages\keras\engine\network.py:872: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 's0:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'c0:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
C:\Python3.5\lib\site-packages\keras\engine\network.py:872: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/TensorArrayReadV3:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_4:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
C:\Python3.5\lib\site-packages\keras\engine\network.py:872: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1_1/TensorArrayReadV3:

Epoch 2/30
 - 48s - loss: 18.1198 - dense_3_loss: 0.2131

Epoch 00002: loss improved from 18.89114 to 18.11977, saving model to model_.h5
Epoch 3/30
 - 50s - loss: 17.7004 - dense_3_loss: 0.1617

Epoch 00003: loss improved from 18.11977 to 17.70044, saving model to model_.h5
Epoch 4/30
 - 48s - loss: 17.3462 - dense_3_loss: 0.1352

Epoch 00004: loss improved from 17.70044 to 17.34623, saving model to model_.h5
Epoch 5/30
 - 48s - loss: 17.0215 - dense_3_loss: 0.1198

Epoch 00005: loss improved from 17.34623 to 17.02146, saving model to model_.h5
Epoch 6/30
 - 49s - loss: 16.7635 - dense_3_loss: 0.1036

Epoch 00006: loss improved from 17.02146 to 16.76354, saving model to model_.h5
Epoch 7/30
 - 45s - loss: 16.5496 - dense_3_loss: 0.0890

Epoch 00007: loss improved from 16.76354 to 16.54960, saving model to model_.h5
Epoch 8/30
 - 46s - loss: 16.3859 - dense_3_loss: 0.0775

Epoch 00008: loss improved from 16.54960 to 16.38594, saving model to model_.h5
Epoch 9/30
 - 47s - loss: 16.2485 

model.load_weights('model_.h5')

In [152]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [153]:
# generate target given source sequence
def predict_sequence(model, tokenizer, source, s0, c0):
    prediction = model.predict([source, s0, c0])
    prediction = (np.array(prediction))
    prediction = prediction.swapaxes(0,1).reshape(prediction.shape[0], prediction.shape[2])

    integers = [argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

In [154]:
def tokens_to_string(self, tokens):
    words = [self.index_to_word[token] for token in tokens if token != 0]

    text = " ".join(words)

    return text

In [155]:
def print_trans(sources, model, tokenizer, s0, c0):
    for i, source in enumerate(sources):
        source_ = source.reshape((1, source.shape[0], source.shape[1]))
        translation = predict_sequence(model, tokenizer, source_, s0, c0)
        print(tokens_to_string(fra_tokenizer, list(np.argmax(source, axis = 1))),"==>",translation)

In [156]:
example = np.argmax(testX, axis = 2)
print(example.shape)
testX.shape

(1000, 10)


(1000, 10, 4397)

In [160]:
example1 = trainX[8990:8999, :]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))

In [161]:
print_trans(example1, model, eng_tokenizer, s0, c0)

il ma laissee tomber ==> i i you
soyons heureux ==> im is
tom sest reveille trop tard ==> im a
je ne suis pas aveugle ==> im is
tout est faux ==> im is
je suis divorce ==> i you
je reviens ==> im is
grand comment ==> i a a
je suis divorcee ==> im you


In [121]:
# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset, s0, c0):
    actual, predicted = list(), list()
    for i, source in enumerate(sources):
        # translate encoded source text
        source_ = source.reshape((1, source.shape[0], source.shape[1]))
        translation = predict_sequence(model, tokenizer, source_, s0, c0)
        raw_target, raw_src = raw_dataset[i]
        actual.append(raw_target.split())
        predicted.append(translation.split())
    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [123]:
# test on some training sequences
print('train')
evaluate_model(model, eng_tokenizer, trainX, train, s0, c0)
# test on some test sequences
print('test')
evaluate_model(model, eng_tokenizer, testX, test, s0, c0)

train


C:\Python3.5\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Python3.5\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


BLEU-1: 0.154473
BLEU-2: 0.002997
BLEU-3: 0.000000
BLEU-4: 0.000000
test
BLEU-1: 0.150281
BLEU-2: 0.009067
BLEU-3: 0.000000
BLEU-4: 0.000000
